# FIXTURE UPDATE  

Atualiza os dados das partidas de uma LIGA em seu respectivo ANO!  

Obrigatorio passar LEAGUE_ID e SESON  
Atualizado:
<ul>
  <li>Referee</li>
  <li>Status</li>
  <li>Fixture</li>
</ul>

> #TODO  
>  
> -> Hoje os IDs dos times entao sendo inseridos da forma que vem, temos que validar se aquele id de time, é o mesmo ID na tabela de **teams**  
> -> Jogar resultset em um dict, tratar e fazer um insert mais limpo  
> .


In [26]:
league = 71
#season - Em caso de ligas que inician em um ano e finalizam em outro, utilizar o ano de inicio
season = 2021

In [27]:
pip install ipython_genutils

Note: you may need to restart the kernel to use updated packages.


In [28]:
import ipynb.fs.full.FootballApi as api
import ipynb.fs.full.PostgreDB as pg
import json
import time
import ipython_genutils
from datetime import datetime

### ATUALIZE AS RODADAS(ROUND)!

In [29]:
# atualizando rounds
#%run ./round_updater.ipynb

185
{'get': 'fixtures/rounds', 'parameters': {'season': '2021', 'league': '71'}, 'errors': [], 'results': 38, 'paging': {'current': 1, 'total': 1}, 'response': ['Regular Season - 1', 'Regular Season - 2', 'Regular Season - 3', 'Regular Season - 4', 'Regular Season - 5', 'Regular Season - 6', 'Regular Season - 7', 'Regular Season - 8', 'Regular Season - 9', 'Regular Season - 10', 'Regular Season - 11', 'Regular Season - 12', 'Regular Season - 13', 'Regular Season - 14', 'Regular Season - 15', 'Regular Season - 16', 'Regular Season - 17', 'Regular Season - 18', 'Regular Season - 19', 'Regular Season - 20', 'Regular Season - 21', 'Regular Season - 22', 'Regular Season - 23', 'Regular Season - 24', 'Regular Season - 25', 'Regular Season - 26', 'Regular Season - 27', 'Regular Season - 28', 'Regular Season - 34', 'Regular Season - 29', 'Regular Season - 31', 'Regular Season - 30', 'Regular Season - 32', 'Regular Season - 33', 'Regular Season - 35', 'Regular Season - 36', 'Regular Season - 37

In [30]:
conn = pg.PostgreDB.connect()

In [31]:
#importando fixture
fixtures = api.FootballApi(function=f"fixtures?league={league}&season={season}").get_data()

In [ ]:

my_json = fixtures['response'][0]

#print(json.dumps(my_json, indent=4))
#print(fixtures['response'][0])

In [ ]:
for fixture in fixtures['response']:

    time.sleep(10)

    fixture_id = fixture["fixture"]['id']
    fixture_referee = fixture["fixture"]['referee']
    fixture_timezone = fixture["fixture"]['timezone']
    fixture_date = fixture["fixture"]['date']
    fixture_timestamp = datetime.utcfromtimestamp(fixture["fixture"]['timestamp'])
    
    fixture_periods = fixture["fixture"]['periods']
    if fixture_periods['first']:
        fixture_first_half_start = datetime.utcfromtimestamp(fixture_periods['first'])
    else:
        fixture_first_half_start = datetime.fromisoformat('1900-01-01T00:00:00')
        
    if fixture_periods['second']:
        fixture_second_half_start = datetime.utcfromtimestamp(fixture_periods['second'])
    else:
        fixture_second_half_start = datetime.fromisoformat('1900-01-01T00:00:00')

    team_home = fixture["teams"]['home']
    team_home_id = team_home['id']
    team_home_name = team_home['name']
    team_home_winner = team_home['winner'] if team_home['winner'] is not None else False
    team_home_goals = fixture["goals"]['home'] if fixture["goals"]['home'] is not None else 0

    team_away = fixture["teams"]['away']
    team_away_id = team_away['id']
    team_away_name = team_away['name']
    team_away_winner = team_away['winner'] if team_away['winner'] is not None else False
    team_away_goals = fixture["goals"]['away'] if fixture["goals"]['away'] is not None else 0

    team_detail_score = fixture["score"]
    team_home_ht_goals = team_detail_score["halftime"]['home'] if team_detail_score["halftime"]['home'] is not None else 0
    team_home_ft_goals = team_detail_score["fulltime"]['home'] if team_detail_score["fulltime"]['home'] is not None else 0
    team_home_extratime_goals = team_detail_score["extratime"]['home'] if team_detail_score["extratime"]['home'] is not None else 0
    team_home_penalty_goals = team_detail_score["penalty"]['home'] if team_detail_score["penalty"]['home'] is not None else 0
    team_away_ht_goals = team_detail_score["halftime"]['away'] if team_detail_score["halftime"]['away'] is not None else 0
    team_away_ft_goals = team_detail_score["fulltime"]['away'] if team_detail_score["fulltime"]['away'] is not None else 0
    team_away_extratime_goals = team_detail_score["extratime"]['away'] if team_detail_score["extratime"]['away'] is not None else 0
    team_away_penalty_goals = team_detail_score["penalty"]['away'] if team_detail_score["penalty"]['away'] is not None else 0

    status = fixture["fixture"]['status']
    fixture_status_long = status['long']
    fixture_status_short = status['short']

    # apenas para validacao
    league_id = fixture["league"]['id'] if fixture["league"]['id'] is not None else 'NULL'
    league_season = fixture["league"]['season'] if fixture["league"]['season'] is not None else 'NULL'
    stadium = fixture["fixture"]['venue']
    stadium_id = stadium['id'] if stadium['id'] is not None else 'NULL'


    # ---------------------------------------------------------------------
    # ROUND ID
    find_round = fixture["league"]['round'] 
    if find_round:
        sql = f"""select id from round
                    where id_league = {league}
                    and season = {season}
                    and name='{find_round}'"""
        #print(sql)
        cur5 = pg.PostgreDB.run_query(conn, sql)
        res5 = cur5.fetchone()
        if res5:
            id_round = res5[0]
        else:
            id_round = 0

    # ---------------------------------------------------------------------
    # REFEREE
    if fixture_referee:
        sql = f"SELECT id FROM referee where name='{fixture_referee}'"
        #print(sql)
        cur1 = pg.PostgreDB.run_query(conn, sql)
        res1 = cur1.fetchone()
        if res1:
            referee_id = res1[0]
        else:
            sql = f"""INSERT INTO referee (name) VALUES('{fixture_referee}') 
            ON CONFLICT DO NOTHING 
            RETURNING id """
            #print(sql)
            cur2 = pg.PostgreDB.run_query(conn, sql)
            res2 = cur2.fetchone()
            if res2:
                referee_id = res2[0]
            else:
                referee_id = 'NULL'
    else:
        referee_id = 'NULL'

    # ---------------------------------------------------------------------
    # STATUS

    if fixture_status_long:
        sql = f"SELECT id FROM status where long_status='{fixture_status_long}'"
        
        cur12 = pg.PostgreDB.run_query(conn, sql)
        res12 = cur12.fetchone()
        if res12:
            status_id = res12[0]
        else:
            sql = f"""INSERT INTO status (long_status, short_status) VALUES('{fixture_status_long}','{fixture_status_short}') 
                    ON CONFLICT DO NOTHING 
                    RETURNING id """
            cur22 = pg.PostgreDB.run_query(conn, sql)
            res22 = cur22.fetchone()
            if res22:
                status_id = res22[0]
            else:
                status_id = 'NULL'
    else:
        status_id = 'NULL'
    
    # ---------------------------------------------------------------------
    # FIXTURE/PARTIDA

    sql = f""" INSERT INTO fixture
                    (id, 
                    id_league,
                    id_round,
                    season, 
                    id_referee, 
                    id_stadium, 
                    schedule_timestamp, 
                    first_period_start_timestamp, 
                    second_period_start_timestamp, 
                    id_status, 
                    id_home_team, 
                    home_winner, 
                    home_goals, 
                    id_away_team, 
                    away_winner, 
                    away_goals, 
                    ht_home_goals,
                    ht_away_goals, 
                    ft_home_goals, 
                    ft_away_goals, 
                    et_home_goals, 
                    et_away_goals, 
                    penalty_home, 
                    penalty_away)
                VALUES({fixture_id}, 
                    {league_id}, 
                    {id_round},
                    {league_season},
                    {referee_id}, 
                    {stadium_id}, 
                    '{fixture_timestamp}', 
                    '{fixture_first_half_start}', 
                    '{fixture_second_half_start}',
                    {status_id}, 
                    {team_home_id}, 
                    {team_home_winner}, 
                    {team_home_goals}, 
                    {team_away_id}, 
                    {team_away_winner}, 
                    {team_away_goals}, 
                    {team_home_ht_goals}, 
                    {team_away_ht_goals}, 
                    {team_home_ft_goals}, 
                    {team_away_ft_goals}, 
                    {team_home_extratime_goals}, 
                    {team_away_extratime_goals}, 
                    {team_home_penalty_goals}, 
                    {team_away_penalty_goals})
                ON CONFLICT DO NOTHING
            """
    #print(sql)
    pg.PostgreDB.run_query(conn, sql)


pg.PostgreDB.commit(conn)
pg.PostgreDB.close(conn)